# [STARTER] Exercise - Building an Agentic RAG System

In this exercise, you will build an Agentic RAG (Retrieval-Augmented Generation) system that 
combines the power of AI agents with traditional RAG pipelines. You'll create an agent that 
can decide when and how to retrieve information from different sources, including vector 
databases, web search, and other tools.


## Challenge

Your challenge is to create an Agentic RAG system that can:

- Build a RAG pipeline as a tool that can be used by the agent
- Create an agent that can decide which tool to use based on the query
- Handle different types of queries intelligently
- Combine information from multiple sources when needed


## Setup
First, let's import the necessary libraries:

In [1]:
from pathlib import Path
import sys

nb_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()
parent = nb_dir.parent  # points to 03-building-agents
if str(parent) not in sys.path:
    sys.path.insert(0, str(parent))

In [2]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import os
from typing import List
from dotenv import load_dotenv

from lib.agents import Agent
from lib.llm import LLM
from lib.state_machine import Run
from lib.messages import BaseMessage
from lib.tooling import tool
from lib.vector_db import VectorStoreManager, CorpusLoaderService
from lib.rag import RAG

In [4]:
load_dotenv()

False

In [5]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Load data to Vector DB

In [6]:
db = VectorStoreManager(OPENAI_API_KEY)
db

VectorStoreManager():<chromadb.api.client.Client object at 0x11da9c050>

In [7]:
loader_service = CorpusLoaderService(db)

In [8]:
rag_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

In [9]:
games_market_rag = RAG(
    llm=rag_llm,
    vector_store=loader_service.load_pdf(
        store_name="games_market",
        pdf_path="TheGamingIndustry2024.pdf",
    )
)

VectorStore `games_market` ready!
Pages from `TheGamingIndustry2024.pdf` added!


In [10]:
result:Run = games_market_rag.invoke(
    "What's the  state of virtual reality"
)
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
The state of virtual reality (VR) in 2024 is characterized by its significant impact on the gaming industry, allowing players to immerse themselves in realistic and interactive digital environments. VR technology is enhancing gaming experiences by providing high-fidelity graphics, intuitive controls, and immersive audio. Games like "Half-Life: Alyx" exemplify this trend, offering players a visceral experience where their decisions have meaningful consequences. Additionally, VR is being utilized beyond gaming, with applications in education, therapy, and mental health, showcasing its transformative potential in various fields. Overall, VR is a key technological innovation that is reshaping how players engage with digital worlds.


In [11]:
electric_vehicles_rag = RAG(
    llm=rag_llm,
    vector_store=loader_service.load_pdf(
        store_name="electric_vehicles",
        pdf_path="GlobalEVOutlook2025.pdf",
    )
)

VectorStore `electric_vehicles` ready!
Pages from `GlobalEVOutlook2025.pdf` added!


In [12]:
result:Run = electric_vehicles_rag.invoke("What was the number of electric car sales and their market share in Brazil in 2024?")
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
In 2024, Brazil had nearly 125,000 electric car sales, which represented a market share of 6.5%.


## Tools

In a simple form, Agentic RAG can act like a router, choosing between multiple external sources to retrieve relevant information. These sources aren't limited to databases, they can also include tools like web search or APIs for services such as Slack or email.

In this case it will choose between two collections.

In [22]:
@tool
def search_global_ev_collection(query: str) -> dict:
    """Query the 'electric_vehicles' vector store and return a structured result.

    Inputs:
        query (str): Natural-language question.

    Returns:
        dict: {
          "collection": "electric_vehicles",
          "answer": str,              # "" if unknown
          "status": "ok"|"no_context" # hints to the agent
        }
    """
    result: Run = electric_vehicles_rag.invoke(query)
    ans = (result.get_final_state() or {}).get("answer") or ""
    status = "ok" if ans and ans.lower() != "i don't know." else "no_context"
    return {"collection": "electric_vehicles", "answer": ans, "status": status}



In [23]:

@tool
def search_games_market_report_collection(query: str) -> dict:
    """Query the 'games_market' vector store and return a structured result.

    Inputs:
        query (str): Natural-language question.

    Returns:
        dict: {
          "collection": "games_market",
          "answer": str,
          "status": "ok"|"no_context"
        }
    """
    result: Run = games_market_rag.invoke(query)
    ans = (result.get_final_state() or {}).get("answer") or ""
    status = "ok" if ans and ans.lower() != "i don't know." else "no_context"
    return {"collection": "games_market", "answer": ans, "status": status}

In [24]:
def build_instructions_with_tools(tools) -> str:
    lines = []
    for t in tools:
        sig = str(getattr(t, "signature", ""))
        desc = getattr(t, "description", "") or ""
        lines.append(f"- {t.name}{sig}: {desc}")
    catalog = "\n".join(lines)

    return (
        "You are an Agentic RAG assistant that selects the best tool for each query.\n"
        "\n"
        "Action policy:\n"
        "- Prefer the EV collection for EV questions; the games collection for gaming/VR questions.\n"
        "- When a tool returns a structured result with status == 'no_context' or answer == '' or 'I don't know.',\n"
        "  you must immediately try the other tool with the same query (or a refined query).\n"
        "- Keep answers concise. When you use a tool, use the tool’s answer field directly.\n"
        "\n"
        "Tools you can use:\n"
        f"{catalog}\n"
    )

In [25]:
tools = [search_global_ev_collection, search_games_market_report_collection]
instructions = build_instructions_with_tools(tools)

agentic_rag = Agent(
    model_name="gpt-4o-mini",
    tools=tools,
    instructions=instructions,
)

In [26]:
def print_messages(messages: List[BaseMessage]):
    for m in messages: 
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run

In [27]:
run_1 = agentic_rag.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
    session_id="oscar",
)

print("\nMessages from run 1:")
messages = run_1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are an Agentic RAG assistant that selects the best tool for each query.

Action policy:
- Prefer the EV collection for EV questions; the games collection for gaming/VR questions.
- When a tool returns a structured result with status == 'no_context' or answer == '' or 'I don't know.',
  you

In [28]:
run_2 = agentic_rag.invoke(
    query= (
        "Which two countries accounted for most of the electric car exports from " 
        "the Asia Pacific region (excluding China) in 2024?"
    ),
    session_id="electric_car",
)

print("\nMessages from run 2:")
messages = run_2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are an Agentic RAG assistant that selects the best tool for each query.

Action policy:
- Prefer the EV collection for EV questions; the games collection for gaming/VR questions.
- When a tool returns a structured result with status == 'no_context' or answer == '' or 'I don't know.',
  you must immediately try the other tool with the same query (or a refined query).
- Keep answers concise. When you use a tool, use the tool’s answer field directly.

Tools you can use:
- search_global_

In [29]:
run_3 = agentic_rag.invoke(
    query= (
        "Why is generative AI seen more as an accelerator than a replacement in game development?"
    ),
    session_id="games",
)

print("\nMessages from run 3:")
messages = run_3.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 3:
 -> (role = system, content = You are an Agentic RAG assistant that selects the best tool for each query.

Action policy:
- Prefer the EV collection for EV questions; the games collection for gaming/VR questions.
- When a tool returns a structured result with status == 'no_context' or answer == '' or 'I don't know.',
  you must immediately try the other tool with the same query (or a refined query).
- Keep answers concise. When you use a tool, use the tool’s answer field directly.

Tools you can use:
- search_global_